In [2]:
# 1 导入包
import torch
from torch import nn
import pandas as pd
import numpy as np
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

d:\Users\17020\miniconda3\envs\proto\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 2 创建model
class BP_Model(torch.nn.Module):
    def __init__(self):
        super(BP_Model,self).__init__()
        
        self.layer1 = nn.Sequential(nn.Linear(104,50),nn.Sigmoid())
        self.layer2 = nn.Sequential(nn.Linear(50,23))

    def forward(self,flow):
        flow = self.layer1(flow)
        flow = self.layer2(flow)
        return flow
 
# 创建和实例化一个整个模型类的对象
model = BP_Model()
# 打印出整个模型
print(model)

BP_Model(
  (layer1): Sequential(
    (0): Linear(in_features=104, out_features=50, bias=True)
    (1): Sigmoid()
  )
  (layer2): Sequential(
    (0): Linear(in_features=50, out_features=23, bias=True)
  )
)


In [19]:
# 3 读取数据集
# dataframe = pd.read_csv("total.csv",header=None).values # 取消第一行作为表头
# train_set = pd.read_csv("train_set.csv",header=None).values
test_set = pd.read_csv("test_set.csv",header=None) .values

# train_set = np.array(train_set[1:,:]).astype(np.float32) # 去掉第一行字段 
test_set = np.array(test_set[1:,:]).astype(np.float32)


C:\Users\jinghan\AppData\Local\Temp\ipykernel_84804\955486019.py:4: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104) have mixed types. Specify dtype option on import or set low_memory=False.
  test_set = pd.read_csv("test_set.csv",header=None) .values


In [20]:
# 4 读标签
# label = dataframe.loc[:,['label']]
# train_label = train_set[:,0]
test_label = test_set[:,0]


# 5 读特征
# train_feature  = train_set[:,1:]
test_feature = test_set[:,1:]


print(type(test_feature))

# 转换成tensor (台式没有GPU 就不转换了)
# train_feature_tensor = torch.tensor(train_feature)
test_feature_tensor = torch.tensor(test_feature)


<class 'numpy.ndarray'>


In [24]:
# 6 对特征进行归一化
test_nor = torch.nn.functional.normalize(test_feature_tensor,dim=0,eps=1e-12, out=None)


print(test_nor[:,1])

print(test_feature.shape)

tensor([0.0017, 0.0017, 0.0017,  ..., 0.0017, 0.0017, 0.0017])
(340208, 104)


In [ ]:
# PyTorch不会隐式地调整输入的形状。因此，
# 我们在线性层前定义了展平层（flatten），来调整网络输入的形状
net = nn.Sequential(nn.Flatten(), nn.Linear(784, 10))

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights)

In [ ]:
# 7 定义损失函数和优化器

loss = nn.CrossEntropyLoss(reduction='none') # 损失函数
optimizer = torch.optim.SGD(model.parameters(),1e-1) # 随机梯度下降

In [ ]:
trainer = torch.optim.SGD(net.parameters(), lr=0.1) # 学习率为0.1的小批量随机梯度下降作为优化算法

In [ ]:
train_data = DataLoader(test_set, batch_size=64, shuffle=True)  # 训练数据
test_data = DataLoader(test_set, batch_size=128, shuffle=False)  # 测试数据

In [ ]:
# 8 开始训练
num_epochs = 100
train_losses = []
train_acces = []
# 用数组保存每一轮迭代中，在测试数据上测试的损失值和精确度，也是为了通过画图展示出来。
eval_losses = []
eval_acces = []

for e in test_feature_tensor:
    train_loss = 0
    train_acc = 0
    model.train()   # 将模型改为训练模式
    # 4.1==========================训练模式==========================
    train_loss = 0
    train_acc = 0
    model.train()   # 将模型改为训练模式

    # 每次迭代都是处理一个小批量的数据，batch_size是64
    for im, label in train_data:
        im = Variable(im)
        label = Variable(label)

        # 计算前向传播，并且得到损失函数的值
        out = model(im)
        loss = criterion(out, label)

        # 反向传播，记得要把上一次的梯度清0，反向传播，并且step更新相应的参数。
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 记录误差
        train_loss += loss.item()

        # 计算分类的准确率
        _, pred = out.max(1)
        num_correct = (pred == label).sum().item()
        acc = num_correct / im.shape[0]
        train_acc += acc

    train_losses.append(train_loss / len(train_data))
    train_acces.append(train_acc / len(train_data))



In [ ]:
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, trainer)